## Fig 1: Dispersion relation

In [58]:
from meclat import MechanicalLattice1D

In [59]:
ml1d = MechanicalLattice1D(k=[.7, 1.3], m=[1.3, .7], precision=0.1)
ws, evs = ml1d.dispersion()

# w0 = ws[:, 0]
# w1 = ws[:, 1]
# print(w0.shape)
# print(w0)
# print(evs[0].shape)
# print(evs[0])

In [60]:
import pandas as pd
import numpy as np

w0 = ws[:, 0]
w1 = ws[:, 1]
ws = np.append(w0, w1)

x = np.append(ml1d.qs, ml1d.qs)
y = ws
index = np.append(np.repeat(0, len(ml1d.qs)), np.repeat(1, len(ml1d.qs)))


df = pd.DataFrame({
    "q": x,
    "w": y,
    "index": index,
})

In [61]:
import plotly.express as px

fig = px.line(df, x="q", y="w", color='index')
fig.show()